# kaggle username and api key to env params and dataset download

In [16]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import TargetEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [17]:
"""
with open("/Users/arsknz/Downloads/kaggle.json") as f:
    us_key = json.load(f)

os.environ["KAGGLE_USERNAME"] = us_key["username"]
os.environ["KAGGLE_KEY"] = us_key["key"]

!kaggle datasets download -d gabrielrichter/sales-by-products-assortment  -p ~/osnov/code/sbor_data/ --force
!unzip sales-by-products-assortment.zip
"""

'\nwith open("/Users/arsknz/Downloads/kaggle.json") as f:\n    us_key = json.load(f)\n\nos.environ["KAGGLE_USERNAME"] = us_key["username"]\nos.environ["KAGGLE_KEY"] = us_key["key"]\n\n!kaggle datasets download -d gabrielrichter/sales-by-products-assortment  -p ~/osnov/code/sbor_data/ --force\n!unzip sales-by-products-assortment.zip\n'

In [18]:
data = pd.read_csv("sales_assortments.csv", index_col="Index")
data.sample(5)

,Date,Sales,Assortment
Index,,,
934,2018 oct 02,85806.394154,drink
1407,2020 dec 11,7167.647393,savory_snacks
500,2017 mar 20,23749.154835,savory_snacks
200,2016 aug 17,43251.904056,drink
1543,2020 jun 17,10317.091401,candy


In [19]:
data.Date = pd.to_datetime(data.Date, format="%Y %b %d")
data

,Date,Sales,Assortment
Index,,,
0,2014-08-01,34201.792962,drink
1,2014-08-02,8404.176909,drink
2,2014-08-03,62851.436278,drink
3,2014-08-04,79310.692652,drink
4,2014-08-05,83689.581215,drink
...,...,...,...
1695,2020-09-26,302.356902,drink
1696,2020-09-27,24966.162859,drink
1697,2020-09-28,12161.923139,drink


In [20]:
def month_to_num(date):
    month_str = date.strftime("%b")
    month_dict = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }
    return month_dict[month_str]

In [21]:
data["Month"] = data["Date"].apply(lambda x: month_to_num(x))
data["IsWeekend"] = data["Date"].dt.dayofweek >= 5
data.sample(5)

,Date,Sales,Assortment,Month,IsWeekend
Index,,,,,
842,2018-03-02,6982.005766,savory_snacks,3,False
839,2018-06-29,1286.222710,candy,6,False
1689,2020-09-20,13513.438336,drink,9,True
1584,2020-05-07,99728.370428,candy,5,False
1576,2020-03-30,85277.847399,candy,3,False


In [22]:
encoder = TargetEncoder(smooth="auto")
data["Assortment_encoded"] = encoder.fit_transform(data[["Assortment"]], data["Sales"])

In [23]:
X = data[["Month", "IsWeekend", "Assortment_encoded"]]
y = data["Sales"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [25]:
model = RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [26]:
mae, r2

(51079.70426370522, 0.24300993585423547)

In [27]:
data

,Date,Sales,Assortment,Month,IsWeekend,Assortment_encoded
Index,,,,,,
0,2014-08-01,34201.792962,drink,8,False,90976.947311
1,2014-08-02,8404.176909,drink,8,True,90976.947311
2,2014-08-03,62851.436278,drink,8,True,93243.353909
3,2014-08-04,79310.692652,drink,8,False,93243.353909
4,2014-08-05,83689.581215,drink,8,False,90976.947311
...,...,...,...,...,...,...
1695,2020-09-26,302.356902,drink,9,True,90976.947311
1696,2020-09-27,24966.162859,drink,9,True,90976.947311
1697,2020-09-28,12161.923139,drink,9,False,95322.930901


In [36]:
ohe = OneHotEncoder(sparse_output=False)
assortment_encoded = ohe.fit_transform(data[["Assortment"]])

In [ ]:
assortment_df = pd.DataFrame(
    assortment_encoded, columns=ohe.get_feature_names_out(["Assortment"])
)

In [44]:
data = pd.concat([data, assortment_df], axis=1)

In [45]:
data

,Date,Sales,Assortment,Month,IsWeekend,Assortment_encoded,Assortment_candy,Assortment_drink,Assortment_savory_snacks
0,2014-08-01,34201.792962,drink,8,False,90976.947311,0.0,1.0,0.0
1,2014-08-02,8404.176909,drink,8,True,90976.947311,0.0,1.0,0.0
2,2014-08-03,62851.436278,drink,8,True,93243.353909,0.0,1.0,0.0
3,2014-08-04,79310.692652,drink,8,False,93243.353909,0.0,1.0,0.0
4,2014-08-05,83689.581215,drink,8,False,90976.947311,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
1695,2020-09-26,302.356902,drink,9,True,90976.947311,0.0,1.0,0.0
1696,2020-09-27,24966.162859,drink,9,True,90976.947311,0.0,1.0,0.0
1697,2020-09-28,12161.923139,drink,9,False,95322.930901,0.0,1.0,0.0
1698,2020-09-29,25187.078681,drink,9,False,98162.661039,0.0,1.0,0.0


In [46]:
X2 = data[["Month", "IsWeekend"] + list(assortment_df.columns)]
y2 = data["Sales"]

In [47]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42
)

In [48]:
model2 = RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42)
model2.fit(X2_train, y2_train)
y2_pred = model2.predict(X2_test)
mae2 = mean_absolute_error(y2_test, y2_pred)
r2_2 = r2_score(y2_test, y2_pred)

In [49]:
mae2, r2_2

(51126.56642101711, 0.23878285866101523)

### Encoding не очень влияет, а вообще по хорошему конечно строить пайплайн для lstm